In [1]:
!conda update -y dask s3fs

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - dask
    - s3fs


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tbb-2019.8                 |       hfd86e86_0         1.4 MB
    ca-certificates-2019.11.27 |                0         132 KB
    dask-2.9.0                 |             py_0          12 KB
    fsspec-0.6.2               |             py_0          50 KB
    s3fs-0.4.0                 |             py_0          24 KB
    distributed-2.9.0          |             py_0         401 KB
    partd-1.1.0                |             py_0          20 KB
    dask-core-2.9.0            |             py_0         5

### Check connectivity to initial client

In [5]:
from dask.distributed import Client
client = Client('Dask-Scheduler.local-dask:8786')
client

Client Scheduler: tcp://Dask-Scheduler.local-dask:8786 Dashboard: http://Dask-Scheduler.local-dask:8787/status,Cluster Workers: 8 Cores: 32 Memory: 14.40 GB


### Increase number of workers

In [50]:
!aws ecs update-service --service Dask-Worker --desired-count 50 --cluster Fargate-Dask-Cluster

{
    "service": {
        "serviceArn": "arn:aws:ecs:us-east-1:497456752804:service/Dask-Worker",
        "serviceName": "Dask-Worker",
        "clusterArn": "arn:aws:ecs:us-east-1:497456752804:cluster/Fargate-Dask-Cluster",
        "loadBalancers": [],
        "serviceRegistries": [
            {
                "registryArn": "arn:aws:servicediscovery:us-east-1:497456752804:service/srv-7litxs3svo5nq4f5"
            }
        ],
        "status": "ACTIVE",
        "desiredCount": 50,
        "runningCount": 25,
        "pendingCount": 0,
        "launchType": "FARGATE",
        "platformVersion": "LATEST",
        "taskDefinition": "arn:aws:ecs:us-east-1:497456752804:task-definition/Dask-Worker:11",
        "deploymentConfiguration": {
            "maximumPercent": 200,
            "minimumHealthyPercent": 100
        },
        "deployments": [
            {
                "id": "ecs-svc/9223370459714451620",
                "status": "PRIMARY",
                "taskDefinition": "a

### Wait for some time before restarting and checking if num Workers = 8

In [51]:
client.restart()

Client Scheduler: tcp://Dask-Scheduler.local-dask:8786 Dashboard: http://Dask-Scheduler.local-dask:8787/status,Cluster Workers: 25 Cores: 100 Memory: 45.00 GB


In [58]:
client

Client Scheduler: tcp://Dask-Scheduler.local-dask:8786 Dashboard: http://Dask-Scheduler.local-dask:8787/status,Cluster Workers: 50 Cores: 200 Memory: 90.00 GB


In [68]:
import dask.array as da
x = da.random.random((100000, 100000), chunks=(500, 2500))
x

,Array,Chunk
Bytes,80.00 GB,50.00 MB
Shape,"(100000, 100000)","(2500, 2500)"
Count,1600 Tasks,1600 Chunks
Type,float64,numpy.ndarray


In [69]:
y = x + x.T
z = y.mean(axis=1)
z

dask.array<mean_agg-aggregate, shape=(100000,), dtype=float64, chunksize=(2500,), chunktype=numpy.ndarray>

In [70]:
%%time
z.compute()

CPU times: user 502 ms, sys: 12.4 ms, total: 515 ms
Wall time: 42.4 s


array([1.0012233 , 1.00018291, 1.00015995, ..., 1.00167455, 1.00202238,
       1.00198984])

8 workers, 32 cores, 14.4 GB 

|Array | Chunk | Time |
|-|-|-|
|10000 | 1000|1.46 sec|
|10000 | 2500|1.40 sec|
|20000 | 1000|4.85 sec|
|20000 | 2500|4.88 sec|
|50000 | 1000|32.6 sec|
|50000 | 2500|49.5 sec|



25 workers, 100 cores, 45 GB 

|Array | Chunk | Time |
|-|-|-|
|50000 | 1000|18.3 sec|
|50000 | 2500|12.3 sec|
|100000 | 1000|29.4 sec|
|100000 | 2500|120 sec|


50 workers, 200 cores, 90 GB 

|Array | Chunk | Time |
|-|-|-|
|50000 | 1000|7.42 sec|
|50000 | 2500|6.06 sec|
|100000 | 1000|57.1 sec|
|100000 | 2500|42.4 sec|




In [71]:
import dask
import dask.dataframe as dd
df = dask.datasets.timeseries()

In [73]:
df2 = df[df.y > 0]
df3 = df2.groupby('name').x.std()

In [74]:
%%time
df3.compute()

CPU times: user 31.5 ms, sys: 4.28 ms, total: 35.7 ms
Wall time: 332 ms


name
Alice       0.576885
Bob         0.578031
Charlie     0.578062
Dan         0.576427
Edith       0.577605
Frank       0.579222
George      0.577573
Hannah      0.576551
Ingrid      0.575477
Jerry       0.576937
Kevin       0.578065
Laura       0.577294
Michael     0.577691
Norbert     0.574096
Oliver      0.578047
Patricia    0.573789
Quinn       0.579780
Ray         0.576359
Sarah       0.576877
Tim         0.578863
Ursula      0.578104
Victor      0.578471
Wendy       0.577900
Xavier      0.576697
Yvonne      0.579133
Zelda       0.578862
Name: x, dtype: float64

In [8]:
import s3fs
import dask.dataframe as dd
import boto3
import dask.distributed

### Read data

We will be testing Dask using https://registry.opendata.aws/noaa-ghcn/#usageexamples

The following information serves as a definition of each field in one line of data covering one station-day. Each field described below is separated by a comma ( , ) and follows the order
presented in this document.

* ID = 11 character station identification code
* YEAR/MONTH/DAY = 8 character date in YYYYMMDD format (e.g. 19860529 = May 29, 1986)
* ELEMENT = 4 character indicator of element type 
* DATA VALUE = 5 character data value for ELEMENT 
* M-FLAG = 1 character Measurement Flag 
* Q-FLAG = 1 character Quality Flag 
* S-FLAG = 1 character Source Flag 
* OBS-TIME = 4-character time of observation in hour-minute format (i.e. 0700 =7:00 am)

See section III of the GHCN-Daily readme.txt file for an explanation of ELEMENT codes and their units as well as the M-FLAG, Q-FLAGS and S-FLAGS.

The OBS-TIME field is populated with the observation times contained in NOAA/NCDC’s Multinetwork Metadata System (MMS).  



In [25]:
cols = [ 'stationid', 'date', 'element', 'obsval', 'mflag','qflag','sflag','obstime']

#  station identifier (GHCN Daily Identification Number)
#  date (yyyymmdd; where yyyy=year; mm=month; and, dd=day)
#  observation type (see ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt for definitions)
#  observation value (see ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt for units)
#  observation time (if available, as hhmm where hh=hour and mm=minutes in local time)
    

df = dd.read_csv('s3://noaa-ghcn-pds/csv/2018.csv',storage_options={'anon':False},names=cols)
df_persisted = client.persist(df)

In [26]:
df_persisted.columns

Index(['stationid', 'date', 'element', 'obsval', 'mflag', 'qflag', 'sflag',
       'obstime'],
      dtype='object')

In [30]:
df.groupby(df_persisted.stationid).obsval.mean().compute()

PermissionError: Access Denied